In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [2]:
## load the ANN trained model
model=load_model('model.h5')

##load the encoder and scaler
with open('onehotencoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)



In [3]:
#example input data
input_data ={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000

}


In [5]:
# One-Hot Encode 'Geography'
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


f:\Projects\DLProject\dl_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
#Encode categorical variable
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:
## 

In [8]:
#combine one-hot encoded columns with input data
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.51811103,  0.91232714,  0.0935768 , -0.69148217, -0.25473337,
         0.80980337,  0.64589732,  0.96655883, -0.87174292,  0.99252792,
        -0.57369368, -0.57523331]])

In [10]:
#prediction churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 15s 15s/step


array([[0.04283392]], dtype=float32)

In [11]:
prediction_prob= prediction[0][0]

In [12]:
prediction_prob

0.04283392

In [13]:
if prediction_prob>0.5:
    print('Churn')
else:
    print('Not Churn')

Not Churn
